### 1. Data hashing time (changes with replica size)

In [1]:
import hashlib # Install blspy using `pip install blspy`
import json
import time

data_blocks = []
for i in range(16):
    data_blocks.append(bytes([i]*1024*1024*64))

data_hashing_times = {}
replica_sizes = [64, 128, 256, 512, 1024]
for replica_size in replica_sizes:
    data = b''.join(data_blocks[:replica_size//64])
    data_hashing_times[replica_size] = []
    for i in range(100):
        t = time.time()
        data_hash = hashlib.sha256(data).digest()
        data_hashing_times[replica_size].append(time.time() - t)

with open('1_data_hashing_times.json', 'w') as f:
    json.dump(data_hashing_times, f)

### 2. BLS signature generation time

In [24]:
import hashlib
import json
import random
import time
from blspy import AugSchemeMPL, PopSchemeMPL

seed = bytes([random.randint(0, 255) for _ in range(32)])
sk = AugSchemeMPL.key_gen(seed)
pk = sk.get_g1()
data = bytes([0]*1024)
data_hash = hashlib.sha256(data).digest()

signature_generation_times = []
for i in range(100):
    t = time.time()
    signature = PopSchemeMPL.sign(sk, data_hash)
    signature_generation_times.append(time.time() - t)

with open('2_signature_generation_times.json', 'w') as f:
    json.dump(signature_generation_times, f)

### 3. BLS signature aggregation time (changes with number of signatures)

In [23]:
import hashlib
import json
import random
import time
from blspy import AugSchemeMPL, PopSchemeMPL

n = 100
signatures = []
data = bytes([0]*1024)
data_hash = hashlib.sha256(data).digest()
for i in range(n):
    seed = bytes([random.randint(0, 255) for _ in range(32)])
    sk = AugSchemeMPL.key_gen(seed)
    signatures.append(PopSchemeMPL.sign(sk, data_hash))

signature_aggregation_times = {}
for i in range(n):
    signature_aggregation_times[i+1] = []
    for j in range(100):
        t = time.time()
        agg_signature = PopSchemeMPL.aggregate(signatures[:i+1])
        signature_aggregation_times[i+1].append(time.time() - t)

with open('3_signature_aggregation_times.json', 'w') as f:
    json.dump(signature_aggregation_times, f)

### 4. BLS signature verification time (changes with number of signatures)

In [4]:
import hashlib
import json
import random
import time
from blspy import AugSchemeMPL, PopSchemeMPL

n = 100
pks = []
signatures = []
data = bytes([0]*1024)
data_hash = hashlib.sha256(data).digest()
for i in range(n):
    seed = bytes([random.randint(0, 255) for _ in range(32)])
    sk = AugSchemeMPL.key_gen(seed)
    pks.append(sk.get_g1())
    signatures.append(PopSchemeMPL.sign(sk, data_hash))

signature_verification_times = {}
for i in range(n):
    signature_verification_times[i+1] = []
    agg_signature = AugSchemeMPL.aggregate(signatures[:i+1])
    for j in range(100):
        t = time.time()
        ok = PopSchemeMPL.fast_aggregate_verify(pks[:i+1], data_hash, 
                                                agg_signature)
        signature_verification_times[i+1].append(time.time() - t)

with open('4_signature_verification_times.json', 'w') as f:
    json.dump(signature_verification_times, f)